<a href="https://colab.research.google.com/github/VasundharaSK/Invoice-Processing/blob/main/invoice_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!git clone https://github.com/zzzDavid/ICDAR-2019-SROIE.git

Cloning into 'ICDAR-2019-SROIE'...
remote: Enumerating objects: 2386, done.
remote: Total 2386 (delta 0), reused 0 (delta 0), pack-reused 2386
Receiving objects: 100% (2386/2386), 278.63 MiB | 26.40 MiB/s, done.
Resolving deltas: 100% (211/211), done.
Checking out files: 100% (1980/1980), done.


In [4]:
## Creating folders for preprocessed dataset
!mkdir boxes_and_transcripts images entities

In [5]:
## Script for preprocessing dataset
import os
import pandas
import json
import csv
import shutil

## Input dataset
data_path = "ICDAR-2019-SROIE/data/"
box_path = data_path + "box/"
img_path = data_path + "img/"
key_path = data_path + "key/"

## Output dataset
out_boxes_and_transcripts = "/content/boxes_and_transcripts/"
out_images = "/content/images/"
out_entities  = "/content/entities/"

In [6]:
train_samples_list =  []
for file in os.listdir(data_path + "box/"):
  
  ## Reading csv
  with open(box_path +file, "r") as fp:
    reader = csv.reader(fp, delimiter=",")
    ## arranging dataframe index ,coordinates x1_1,y1_1,x2_1,y2_1,x3_1,y3_1,x4_1,y4_1, transcript
    rows = [[1] + x[:8] + [','.join(x[8:]).strip(',')] for x in reader] 
    df = pandas.DataFrame(rows)
    ## including ner label dataframe index ,coordinates x1_1,y1_1,x2_1,y2_1,x3_1,y3_1,x4_1,y4_1, transcript , ner tag
  df[10] = 'other'  
  
  ##saving file into new dataset folder
  jpg = file.replace(".csv",".jpg")
  entities = json.load(open(key_path+file.replace(".csv",".json")))
  for key,value in sorted(entities.items()):
    idx = df[df[9].str.contains('|'.join(map(str.strip, value.split(','))))].index
    df.loc[idx, 10] = key

  shutil.copy(img_path +jpg, out_images)
  with open(out_entities + file.replace(".csv",".txt"),"w") as j:  
    print(json.dumps(entities), file=j)
  
  df.to_csv(out_boxes_and_transcripts+file.replace(".csv",".tsv"),index=False,header=False, quotechar='',escapechar='\\',quoting=csv.QUOTE_NONE, )
  train_samples_list.append(['receipt',file.replace('.csv','')])
train_samples_list = pandas.DataFrame(train_samples_list)
train_samples_list.to_csv("train_samples_list.csv")

/usr/local/lib/python3.7/dist-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


#add tag to tcsv file
index ,x1_1,y1_1,x2_1,y2_1,x3_1,y3_1,x4_1,y4_1, transcript , ner tag
1,72,25,326,25,326,64,72,64,TAN WOON YANN,other
1,50,82,440,82,440,121,50,121,BOOK TA .K(TAMAN DAYA) SDN BND,address
1,205,121,285,121,285,139,205,139,789417-W,other
1,110,144,383,144,383,163,110,163,NO.53 55\,57 & 59\, JALAN SAGU 18,address
1,192,169,299,169,299,187,192,187,TAMAN DAYA,address
1,162,193,334,193,334,211,162,211,81100 JOHOR BAHRU,address
1,217,216,275,216,275,233,217,233,JOHOR.,address
1,50,342,279,342,279,359,50,359,DOCUMENT NO : TD01167104,other
1,50,372,96,372,96,390,50,390,DATE:,other
1,165,372,342,372,342,389,165,389,25/12/2018 8:13:39 PM,date 

In [7]:
## document_type, file_name
train_samples_list.head()

,0,1
0,receipt,212
1,receipt,300
2,receipt,617
3,receipt,253
4,receipt,245


In [8]:
#Spliting dataset into train-test sets
from sklearn.model_selection import train_test_split
train_test = pandas.read_csv("train_samples_list.csv",dtype=str)
train, test= train_test_split(train_test,test_size=0.2,random_state = 42)

In [9]:
!git clone https://github.com/wenwenyu/PICK-pytorch.git #original repo
## If you face any problem in training then clone my repo 
## which i have used while making this blog
# !git clone https://github.com/dlmade/Pick.Pytorch.Sroie.git #my repo

Cloning into 'PICK-pytorch'...
remote: Enumerating objects: 293, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 293 (delta 19), reused 24 (delta 10), pack-reused 256
Receiving objects: 100% (293/293), 9.99 MiB | 16.43 MiB/s, done.
Resolving deltas: 100% (134/134), done.


**Copy train data into PICK-pytorch data folder**

In [10]:
for index, row in train.iterrows():
  shutil.copy(out_boxes_and_transcripts+str(row[2])+".tsv",'/content/PICK-pytorch/data/data_examples_root/boxes_and_transcripts/')
  shutil.copy(out_images+str(row[2])+".jpg",'/content/PICK-pytorch/data/data_examples_root/images/')
  shutil.copy(out_entities +str(row[2])+".txt", '/content/PICK-pytorch/data/data_examples_root/entities/')

train.drop(['Unnamed: 0'], axis = 1,inplace = True)
train.reset_index(inplace= True)
train.drop(['index'], axis = 1,inplace = True)
train.to_csv("/content/PICK-pytorch/data/data_examples_root/train_samples_list.csv",header = False)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Copy test data into PICK-pytorch data folder

In [11]:
!mkdir '/content/PICK-pytorch/data/test_data_example/entities/'

In [12]:
for index, row in test.iterrows():
  shutil.copy(out_boxes_and_transcripts+str(row[2])+".tsv",'/content/PICK-pytorch/data/test_data_example/boxes_and_transcripts/')
  shutil.copy(out_images+str(row[2])+".jpg",'/content/PICK-pytorch/data/test_data_example/images/')
  shutil.copy(out_entities +str(row[2])+".txt", '/content/PICK-pytorch/data/test_data_example/entities/')

test.drop(['Unnamed: 0'], axis = 1,inplace = True)
test.reset_index(inplace= True)
test.drop(['index'], axis = 1,inplace = True)
test.to_csv("/content/PICK-pytorch/data/test_data_example/test_samples_list.csv",header = False)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [13]:
## Removing data once it is copied into PICK-pytorch data folder
!rm /content/boxes_and_transcripts/*.tsv
!rm /content/images/*.jpg
!rm /content/entities/*.txt

In [14]:
%cd PICK-pytorch/

/content/PICK-pytorch


In [15]:
%%writefile utils/entities_list.py
# -*- coding: utf-8 -*-
# @Author: Wenwen Yu
# @Created Time: 7/8/2020 9:34 PM

Entities_list = [
    "company",
    "address",
    "date",
    "total"
]

Overwriting utils/entities_list.py


In [16]:
## Installing requirements for running PICK-pytorch
!pip install -r requirements.txt
!pip install torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

     |████████████████████████████████| 6.6MB 3.8MB/s 
     |████████████████████████████████| 43.7MB 70kB/s 
     |████████████████████████████████| 17.3MB 208kB/s 
     |████████████████████████████████| 10.1MB 40.1MB/s 
     |████████████████████████████████| 481kB 39.0MB/s 
     |████████████████████████████████| 71kB 208kB/s 
     |████████████████████████████████| 71kB 8.2MB/s 
     |████████████████████████████████| 753.2MB 20kB/s 
     |████████████████████████████████| 266kB 35.1MB/s 
     |████████████████████████████████| 122kB 41.0MB/s 
     |████████████████████████████████| 133kB 42.2MB/s 
     |████████████████████████████████| 675kB 36.8MB/s 
     |████████████████████████████████| 1.2MB 36.4MB/s 
     |████████████████████████████████| 81kB 9.4MB/s 
     |████████████████████████████████| 7.5MB 38.2MB/s 
     |████████████████████████████████| 5.6MB 32.5MB/s 
     |████████████████████████████████| 901kB 38.1MB/s 
  Created wheel for overrides: filename=overrides-3.0.0

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 704.4MB 26kB/s 
     |████████████████████████████████| 6.7MB 27.1MB/s 
  Found existing installation: torch 1.5.1
    Uninstalling torch-1.5.1:
      Successfully uninstalled torch-1.5.1
  Found existing installation: torchvision 0.6.1
    Uninstalling torchvision-0.6.1:
      Successfully uninstalled torchvision-0.6.1


In [17]:
%%writefile config.json

{
    "name": "PICK_Default",
    "run_id":"test",

    "local_world_size":4,
    "local_rank":-1,
    "distributed":-1,

    "model_arch": {
        "type": "PICKModel",
        "args": {
            "embedding_kwargs":{
                "num_embeddings": -1,
                "embedding_dim": 512
            },
            "encoder_kwargs":{
                "char_embedding_dim":-1,
                "out_dim": 512,
                "nheaders": 4,
                "nlayers": 3,
                "feedforward_dim": 1024,
                "dropout": 0.1,
                "image_encoder": "resnet50",
                "roi_pooling_mode": "roi_align",
                "roi_pooling_size": [7,7]
            },
            "graph_kwargs":{
                "in_dim":-1,
                "out_dim":-1,
                "eta": 1,
                "gamma": 1,
                "learning_dim": 128,
                "num_layers": 2
            },
            "decoder_kwargs":{
                "bilstm_kwargs":{
                    "input_size": -1,
                     "hidden_size": 512,
                     "num_layers": 2,
                     "dropout": 0.1,
                     "bidirectional": true,
                     "batch_first": true

                },
                "mlp_kwargs":{
                     "in_dim": -1,
                     "out_dim": -1,
                    "dropout": 0.1
                },
                "crf_kwargs":{
                    "num_tags":-1
                }
            }
        }
    },

    "train_dataset": {
        "type": "PICKDataset",
        "args": {
            "files_name":"/content/PICK-pytorch/data/data_examples_root/train_samples_list.csv",
            "boxes_and_transcripts_folder":"/content/PICK-pytorch/data/data_examples_root/boxes_and_transcripts",
            "images_folder":"/content/PICK-pytorch/data/data_examples_root/images",
            "entities_folder":"/content/PICK-pytorch/data/data_examples_root/entities",
            "iob_tagging_type":"box_and_within_box_level",
            "resized_image_size": [480, 960],
            "ignore_error": false
        }
    },
    "validation_dataset": {
        "type": "PICKDataset",
        "args": {
            "files_name":"/content/PICK-pytorch/data/test_data_example/test_samples_list.csv",
            "boxes_and_transcripts_folder":"/content/PICK-pytorch/data/test_data_example/boxes_and_transcripts",
            "images_folder":"/content/PICK-pytorch/data/test_data_example/images",
            "entities_folder":"/content/PICK-pytorch/data/test_data_example/entities",
            "iob_tagging_type":"box_and_within_box_level",
            "resized_image_size": [480, 960],
            "ignore_error": false
        }
    },
    "train_data_loader": {
        "type": "DataLoader",
        "args":{
            "batch_size": 4,
            "shuffle": true,
            "drop_last": true,
            "num_workers": 8,
            "pin_memory":true
        }
    },
    "val_data_loader": {
          "type": "DataLoader",
          "args":{
              "batch_size": 4,
              "shuffle": false,
              "drop_last": false,
              "num_workers": 8,
              "pin_memory":true
          }
      },

    "optimizer": {
          "type": "Adam",
          "args":{
              "lr": 0.0001,
              "weight_decay": 0,
              "amsgrad": true
          }
    },
    "lr_scheduler": {
        "type": "StepLR",
        "args": {
            "step_size": 30,
            "gamma": 0.1
        }
    },

    "trainer": {
        "epochs": 100,
        "gl_loss_lambda": 0.01,
        "log_step_interval": 10,
        "val_step_interval": 50,

        "save_dir": "saved/",
        "save_period": 20,
        "log_verbosity": 2,

        "monitor": "max overall-mEF",
        "monitor_open": true,
        "early_stop": 40,

        "anomaly_detection": false,
        "tensorboard": false,

        "sync_batch_norm":true
    }
}

Overwriting config.json


Training

Train atleast 100 epoch for better results.

In [ ]:
#!/bin/bash
!python -m torch.distributed.launch --nnode=1 --node_rank=0 --nproc_per_node=1 \
   train.py -c config.json -d 0 --local_world_size 1
  # --resume /content/PICK-pytorch/saved/models/PICK_Default/test_0917_074722/model_best.pth
   ##uncomment for resume training

2021-05-01 12:40:32.033012: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
[2021-05-01 12:40:36,061 - train - INFO] - Distributed GPU training model start...
[2021-05-01 12:40:36,062 - train - INFO] - [Process 281] Initializing process group with: {'MASTER_ADDR': '127.0.0.1', 'MASTER_PORT': '29500', 'RANK': '0', 'WORLD_SIZE': '1'}
[2021-05-01 12:40:36,063 - train - INFO] - [Process 281] world_size = 1, rank = 0, backend=nccl
[2021-05-01 12:40:36,077 - train - INFO] - Dataloader instances created. Train datasets: 500 samples Validation datasets: 126 samples.
[2021-05-01 12:40:37,089 - train - INFO] - Model created, trainable parameters: 68567386.
[2021-05-01 12:40:37,091 - train - INFO] - Optimizer and lr_scheduler created.
[2021-05-01 12:40:37,091 - train - INFO] - Max_epochs: 100 Log_per_step: 10 Validation_per_step: 50.
[2021-05-01 12:40:37,092 - train - INFO] - Training start...
[2021-05-01 12:40:37,095 - trainer

Testing 

create folder for testing

In [ ]:
##creating testing folders
!mkdir /content/test_img /content/test_boxes_and_transcripts

## copy one file from test sample
import os
import shutil
data_path = "data/test_data_example/boxes_and_transcripts/"
image_path = "data/test_data_example/images/"

out_img_path = "/content/test_img/"
out_box_path = "/content/test_boxes_and_transcripts/"

for file in os.listdir(data_path)[:10]:
  shutil.copy(data_path+file,out_box_path)
  shutil.copy(image_path+file.replace(".tsv",".jpg"),out_img_path)

In [ ]:
## change model_best.pth path
!python test.py --checkpoint saved/models/PICK_Default/test_1003_053713/model_best.pth \
                --boxes_transcripts {out_box_path} \
                --images_path {out_img_path} --output_folder /content/output/ \
                --gpu 0 --batch_size 2